## Direct Implementation

In [2]:
# ==============================================================================
# Baselines – Thesis Evaluation Pipeline (Separat pro Modell)
# ==============================================================================
# Dieses Notebook integriert einfache Baselines (RW, AR1, Mean, MA60)
# in das Stage A/B Framework.
#
# Ziel:
#   - Für JEDE Baseline einen eigenen Modellnamen (MODEL_NAME),
#   - eigene Stage-A- und Stage-B-Ordner,
#   - identische Log-Struktur wie bei den anderen Modellen:
#       * outputs/stageA/baseline_<...>/...
#       * outputs/stageB/baseline_<...>/monthly/preds.csv
# ==============================================================================

import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from statsmodels.tsa.ar_model import AutoReg

# --- 1) Pfad-Setup -----------------------------------------------------------

def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(6):
        if (cur / "src").exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT =", PROJECT_ROOT)

# --- 2) Projekt-Imports ------------------------------------------------------

from src.config import GlobalConfig, outputs_for_model
from src.tuning import run_stageA, run_stageB
from src.io_timesplits import load_target, load_ifo_features

# --- 3) Baseline-Model-Wrapper ----------------------------------------------

class ForecastModel:
    """
    Wrapper für einfache Zeitreihen-Baselines, kompatibel mit der Tuning-Pipeline.

    Unterstützte Strategien (params['strategy']):
      - 'RW'   : Random Walk auf ΔIP -> Vorhersage = 0
      - 'Mean' : Expanding Grand Mean von y
      - 'MA60' : Rolling Mean der letzten 60 Beobachtungen (Fallback: Gesamtmean)
      - 'AR1'  : AR(1) mit Konstante via statsmodels.AutoReg

    API (wie ET/EN/LGBM):
      - fit(X, y, sample_weight=None)
      - predict(X)
      - predict_one(x_row)
      - get_name()
      - get_feature_importances()
    """

    def __init__(self, params=None):
        self.params = params or {}
        self.strategy = self.params.get("strategy", "RW")
        self._backend_name = f"baseline_{self.strategy}"

        # State
        self.y_train_ = None
        self.prediction_ = 0.0  # Skalarer Forecast (nächster Schritt)

    def fit(self, X, y, sample_weight=None):
        """
        Baselines nutzen ausschließlich die Zielhistorie y.
        X und sample_weight werden ignoriert (für kompatible Signatur).
        """
        y_arr = np.asarray(y, dtype=float).ravel()
        self.y_train_ = y_arr

        if self.strategy == "RW":
            # Random Walk auf ΔIP: E[y_{t+1} | F_t] = 0
            self.prediction_ = 0.0

        elif self.strategy == "Mean":
            # Expanding Grand Mean
            self.prediction_ = float(np.mean(y_arr))

        elif self.strategy == "MA60":
            # Rolling Mean über die letzten 60 Beobachtungen
            if len(y_arr) >= 60:
                self.prediction_ = float(np.mean(y_arr[-60:]))
            else:
                # Fallback: Expanding Mean
                self.prediction_ = float(np.mean(y_arr))

        elif self.strategy == "AR1":
            # AR(1) mit Konstante via statsmodels
            try:
                # trend='c' -> Konstante + Lag
                model = AutoReg(y_arr, lags=1, trend="c", old_names=False).fit()
                # Forecast für den nächsten Schritt (t+1)
                self.prediction_ = float(model.forecast(steps=1)[0])
            except Exception:
                # Fallback bei sehr kurzer Historie oder numerischen Problemen
                self.prediction_ = float(np.mean(y_arr))

        else:
            raise ValueError(f"Unbekannte Baseline-Strategie: {self.strategy}")

        return self

    def predict(self, X):
        """
        Gibt für jede Zeile in X denselben skalarer Forecast zurück.
        X wird nur genutzt, um n_samples zu bestimmen.
        """
        if hasattr(X, "shape"):
            n = X.shape[0]
        else:
            n = len(X)
        return np.full(n, self.prediction_, dtype=float)

    def predict_one(self, x_row):
        """Kompatibler Ein-Punkt-Wrapper."""
        return float(self.prediction_)

    def get_name(self):
        return self._backend_name

    def get_feature_importances(self):
        """Baselines haben per Definition keine Feature-Importances."""
        return {}

# --- 4) Daten laden & Konfiguration -----------------------------------------

# Daten nur einmal laden & alignen
y = load_target()
X_ifo = load_ifo_features()  # Dient nur als Dummy-Designmatrix für die Pipeline

# Gemeinsame Zeitachse erzwingen
idx_common = y.index.intersection(X_ifo.index)
y = y.loc[idx_common]
X_ifo = X_ifo.loc[idx_common]

print(f"Daten geladen. T = {len(y)} Beobachtungen.")

def get_thesis_cfg() -> GlobalConfig:
    """
    Lädt die Standard-Thesis-Konfiguration (Splits, Policy etc.).
    """
    cfg = GlobalConfig(preset="thesis")
    return cfg

cfg_obj = get_thesis_cfg()

# --- 5) Baseline-Strategien separat laufen lassen ---------------------------

strategies = ["RW", "Mean", "MA60", "AR1"]

for strat in strategies:
    MODEL_NAME = f"baseline_{strat.lower()}"
    print("\n" + "=" * 80)
    print(f"Starte Baseline: {strat}  (MODEL_NAME = '{MODEL_NAME}')")
    print("=" * 80)

    # Output-Ordner für dieses Baseline-Modell initialisieren
    outputs_for_model(MODEL_NAME)

    # Grid mit genau EINER Konfiguration (nur diese Strategie)
    model_grid = [{"strategy": strat, "seed": 42}]

    # --- Stage A: Dummy-Pass für Shortlist & Stage-A-Logs -------------------
    # Wir behalten diese eine Konfiguration immer bei.
    shortlist = run_stageA(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        model_grid=model_grid,
        X=X_ifo,
        y=y,
        cfg=cfg_obj,
        keep_top_k_final=1,        # exakt 1 Config behalten
        min_survivors_per_block=1  # nie rauswerfen
    )

    # --- Stage B: Finale Evaluierung für diese eine Baseline ----------------
    run_stageB(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        shortlist=shortlist,
        X=X_ifo,
        y=y,
        cfg=cfg_obj
    )

    print(f"Fertig für {strat}.")
    print(f"StageA-Outputs: outputs/stageA/{MODEL_NAME}/")
    print(f"StageB-Outputs: outputs/stageB/{MODEL_NAME}/monthly/preds.csv")

print("\nAlle Baselines fertig.")


PROJECT_ROOT = /Users/jonasschernich/Documents/Masterarbeit/Code
INFO in load_ifo_features: Renaming columns to ensure validity.
Daten geladen. T = 407 Beobachtungen.

Starte Baseline: RW  (MODEL_NAME = 'baseline_rw')
[Stage A] Using FULL FE (Gleis 1 & 2) pipeline.
[Stage A][Block 1] train_end=180, OOS=181-200 | configs=1
  - Config 1/1
    · Month 5/20 processed | running...RMSE=1.6761
    · Month 10/20 processed | running...RMSE=1.3459
    · Month 15/20 processed | running...RMSE=1.2615
    · Month 20/20 processed | running...RMSE=1.1305
[Stage A][Block 1] kept 1 configs (floor=1).
[Stage A][Block 2] train_end=200, OOS=201-220 | configs=1
  - Config 1/1
    · Month 5/20 processed | running...RMSE=0.9028
    · Month 10/20 processed | running...RMSE=1.2099
    · Month 15/20 processed | running...RMSE=2.5466
    · Month 20/20 processed | running...RMSE=2.5601
[Stage A][Block 2] kept 1 configs (floor=1).
[Stage A][Block 3] train_end=220, OOS=221-240 | configs=1
  - Config 1/1
    · Month